In [ ]:
import numpy as np
import pandas as pd
import torch
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Data tokenization

In [ ]:
train = pd.read_json('../input/text-classification/github/embold_train.json')
train_extra = pd.read_json('../input/text-classification/github/embold_train_extra.json')
train = pd.concat([train, train_extra], ignore_index=True)
test = pd.read_json('../input/text-classification/github/embold_test.json')
# 0: bug, 1: feature, 2: question

In [ ]:
train['text'] = train.title + ' ' + train.body
test['text'] = test.title + ' ' + test.body

train.head()

In [ ]:
print(len(train), len(test))

In [ ]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(train, test_size=.2)

In [ ]:
train_texts = train[:20_000].text.to_list()#to_numpy()
val_texts = val[:15_000].text.to_list()#to_numpy()
test_texts = val[-15_000:].text.to_list()
#test_texts = test[:2000].text.to_list()#to_numpy()

In [ ]:
train_labels = torch.tensor(train[:20_000].label.to_list()).to(device)#to_numpy()
val_labels = torch.tensor(val[:15_000].label.to_list()).to(device)#to_numpy()
test_labels = torch.tensor(val[-15_000:].label.to_list()).to(device)
#test_labels = test[:2000].label.to_list()#to_numpy()

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
# If the ``encoded_inputs`` passed are dictionary of numpy arrays, PyTorch tensors or TensorFlow tensors, the
# result will use the same type unless you provide a different tensor type with ``return_tensors``. In the
# case of PyTorch tensors, you will lose the specific device of your tensors however.
# → les texts n'étant pas des tensors, tokenizer ne renvoyait pas de tensors, et donc le to(device) ne fonctionnait pas
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors="pt").to(device)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, return_tensors="pt").to(device)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors="pt").to(device)

# Datasets

In [ ]:
# element_size in bytes
train_encodings.input_ids.element_size()*train_encodings.input_ids.nelement() /10**6

In [ ]:
import torch

class GitDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = GitDataset(train_encodings, train_labels)
val_dataset = GitDataset(val_encodings, val_labels)
test_dataset = GitDataset(test_encodings, test_labels)

# Fine-tuning

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    #precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        #'f1': f1,
        #'precision': precision,
        #'recall': recall
    }


In [ ]:
#model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    #logging_dir='./logs',            # directory for storing logs
    #logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset
    compute_metrics = compute_metrics
)

In [ ]:
model.to(device)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model('bert_github_kaggle')

In [ ]:
print("End")